In [29]:
import os
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams
from dotenv import load_dotenv
from swarmauri.documents.concrete.Document import Document
from swarmauri.conversations.concrete.Conversation import Conversation
from swarmauri.chunkers.concrete.SentenceChunker import SentenceChunker
from swarmauri.vector_stores.concrete.TfidfVectorStore import TfidfVectorStore

In [30]:
# Load environment variables
load_dotenv()

# Retrieve the API key from environment variables
API_KEY = os.getenv("GROQ_API_KEY")

# Check if the API key is set
if not API_KEY:
    print("API key is not set. Please set the GROQ_API_KEY environment variable.")

In [56]:
# Initialize the chunker
vector_store = TfidfVectorStore()
pdf_path = "Resume_CV-Nilay Kumar.pdf"
formatted_text = extract_text_from_pdf(pdf_path)
chunks = chunk_text(formatted_text, chunk_size=300, overlap=150)
documents = [Document(content=chunk) for chunk in chunks]
vector_store.add_documents(documents)

In [63]:
documents

[Document(name=None, id='f2b5e2a9-e6bb-4fa9-be7a-638e2514caf4', members=[], owner=None, host=None, resource='Document', version='0.1.0', type='Document', content='nnilayy.work@gmail.com | LinkedIn | github.com/nnilayy | Certifications | +91-6377778313 Nilay Kumar SUMMARY Passionate Software Engineer with expertise in Full-Stack Development and Machine Learning, with 1+ years of industrial experi- ence at SONY. Skilled in building and deploying scalable web applications, integrating intuitive front-end designs with efficient and secure back-end systems. Research experience spans ML projects under prestigious universities, with a focus on EDA, data prepro- cessing, and developing practical ML solutions. Proficient in designing APIs, database management, and leveraging cloud platforms for application deployment. Committed to continuous learning and adopting best practices in Full-Stack Development and Software Engineering. SKILLS (cid:35) Languages: Python, C, C++, C#, Java ML, DL & Data 

In [66]:
from swarmauri.llms.concrete.GroqModel import GroqModel as LLM
llm = LLM(api_key=API_KEY)
allowed_models = get_allowed_models(llm)
llm.name = allowed_models[2]
llm

GroqModel(name='llama-3.1-70b-versatile', id='68200eb5-4d23-4d9e-8126-f7c426ec44e5', members=[], owner=None, host=None, resource='LLM', version='0.1.0', type='GroqModel', allowed_models=['gemma-7b-it', 'gemma2-9b-it', 'llama-3.1-70b-versatile', 'llama-3.1-8b-instant', 'llama-3.2-11b-text-preview', 'llama-3.2-1b-preview', 'llama-3.2-3b-preview', 'llama-3.2-90b-text-preview', 'llama-guard-3-8b', 'llama3-70b-8192', 'llama3-8b-8192', 'llama3-groq-70b-8192-tool-use-preview', 'llama3-groq-8b-8192-tool-use-preview', 'llava-v1.5-7b-4096-preview', 'mixtral-8x7b-32768'], api_key='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')

In [72]:
from swarmauri.conversations.concrete.MaxSystemContextConversation import MaxSystemContextConversation
from swarmauri.messages.concrete.SystemMessage import SystemMessage
from swarmauri.messages.concrete.HumanMessage import HumanMessage
from swarmauri.agents.concrete.RagAgent import RagAgent

# Create a new system context for the RAG Agent
rag_system_context = "You are an assistant that provides answers to the user. You utilize the details below:"
rag_conversation = MaxSystemContextConversation(
    system_context=SystemMessage(content=rag_system_context), max_size=50
)

rag_agent = RagAgent(
    llm=llm,
    conversation=rag_conversation,
    system_context=rag_system_context,
    vector_store=vector_store,
)

In [71]:
query = "What is their personal email link?"
response = rag_agent.exec(query)
print(response)


Their personal email link is nnilayy.work@gmail.com
